In [1]:
import numpy as np
import pandas as pd 
from sklearn.metrics import (confusion_matrix, precision_recall_curve, auc,
                             roc_curve, recall_score, classification_report, f1_score,
                             precision_recall_fscore_support, ConfusionMatrixDisplay)
import joblib
import os
os.chdir('C:/Users/Marc/Desktop/TFG/R Files/Anomaly Detection for SPC')
from Src.FeatureExt.ML_AD_Preprocessing import *
from sklearn.preprocessing import StandardScaler

#import data and ML Modelss
TS = pd.read_csv("Data/TimeSeries2.csv", index_col = 0)
TS_Class = TS.iloc[:,5]
TS=TS.iloc[:,0]

TS.head()

0    59.424230
1    63.162394
2    35.986608
3    50.691021
4    50.479228
Name: F1, dtype: float64

In [2]:
TSstd=np.array(TS.iloc[0:2000]).reshape(-1,1)
TS=np.array(TS).reshape(-1,1)
scaler = StandardScaler()
scaler = scaler.fit(TS)
X_train = scaler.transform(TS)

X_train[0:5]

array([[0.42410905],
       [0.48688564],
       [0.03051099],
       [0.27744851],
       [0.27389179]])

In [26]:
#Creating sliding windows for each TS and get features
X_test1 = np.transpose(sw_dataset_1(np.transpose(X_train), 20))
scaler = StandardScaler()
scaler = scaler.fit(X_test1)
X_test1 = scaler.transform(X_test1)
X_test1 = pd.DataFrame(X_test1, columns = ["mean20", "sigma20"])

X_test2 = np.transpose(sw_dataset_2(np.transpose(X_train), 20))
scaler = StandardScaler()
scaler = scaler.fit(X_test2)
X_test2 = scaler.transform(X_test2)
X_test2 = pd.DataFrame(X_test2,\
     columns = ["last_value", "mean20", "sigma20","mean5", "sigma5", "find_if", "kurtosis"])

X_test3 = np.transpose(sw_dataset_3(np.transpose(X_train), 20))
scaler = StandardScaler()
scaler = scaler.fit(X_test3)
X_test3 = scaler.transform(X_test3)
X_test3 = pd.DataFrame(X_test3,\
     columns = ["last_value", "mean20", "sigma20","mean5", "sigma5", "find_if", "kurtosis",\
           "dir_change"])

iforest_1 = joblib.load('ML Models/ML_iforest_1.sav')
iforest_2 = joblib.load('ML Models/ML_iforest_2.sav')
iforest_3 = joblib.load('ML Models/ML_iforest_3.sav')
svm_1 = joblib.load('ML Models/ML_svm_1.sav')
svm_2 = joblib.load('ML Models/ML_svm_2.sav')
svm_3 = joblib.load('ML Models/ML_svm_3.sav')

In [27]:
#predict from ML Modelss
#Predict iForest
predictions_forest1 = iforest_1.predict(np.array(X_test1))
predictions_forest1 = np.array((predictions_forest1 == -1)*1)

predictions_forest2 = iforest_2.predict(np.array(X_test2))
predictions_forest2 = np.array((predictions_forest2 == -1)*1)

predictions_forest3 = iforest_3.predict(np.array(X_test3))
predictions_forest3 = np.array((predictions_forest3 == -1)*1)

#Predict SVM
predictions_SVM1 = svm_1.predict(np.array(X_test1))
predictions_SVM2 = svm_2.predict(np.array(X_test2))
predictions_SVM3 = svm_3.predict(np.array(X_test3))


In [28]:
TS_Class=TS_Class[20:]
suma=np.sum(TS_Class)
#Confusion Matrices Global
cm_forest1 = confusion_matrix(TS_Class, predictions_forest1)
cm_forest2 = confusion_matrix(TS_Class, predictions_forest2)
cm_forest3 = confusion_matrix(TS_Class, predictions_forest3)
cm_SVM1 = confusion_matrix(TS_Class, predictions_SVM1)
cm_SVM2 = confusion_matrix(TS_Class, predictions_SVM2)
cm_SVM3 = confusion_matrix(TS_Class, predictions_SVM3)

In [48]:
import plotly.express as px

px.scatter(TS, x=np.arange(0,9980), y=TS[20:,0],
           color=predictions_forest3.astype('str'), opacity = 0.2
          )